# Data Wrangling for addiction data set

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer


filename = "mobile_addiction.csv"

In [2]:
# Python list headers containing name of headers
headers = [
    "Unnamed: 0",
    "daily_screen_time",
    "app_sessions",
    "social_media_usage",
    "gaming_time",
    "notifications",
    "night_usage",
    "age",
    "work_study_hours",
    "stress_level",
    "apps_installed",
    "addicted"
]


In [3]:
# Pandas method read_csv() to load the data
# Set the parameter "names"

original = pd.read_csv(filename)
df = original.copy()

In [4]:
# To see what the data set looks like, we'll use the head() method.


  # the first five rows of the dataframe.
df.drop(columns=["Unnamed: 0"], inplace=True)
df.head()
# Drop the ID column as it is not needed for analysis

,daily_screen_time,app_sessions,social_media_usage,gaming_time,notifications,night_usage,age,work_study_hours,stress_level,apps_installed,addicted
0,2,29,0,0,49,0,44,5,3,35,not addicted
1,6,29,1,2,65,1,29,5,9,21,addicted
2,9,28,2,0,57,3,28,7,5,39,addicted
3,6,39,2,0,69,1,28,6,8,24,addicted
4,5,37,3,1,64,2,27,4,5,26,addicted


In [5]:
print(df.shape)
print(df.size)
print(df.dtypes)

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()


(13589, 11)
149479
daily_screen_time      int64
app_sessions           int64
social_media_usage     int64
gaming_time            int64
notifications          int64
night_usage            int64
age                    int64
work_study_hours       int64
stress_level           int64
apps_installed         int64
addicted              object
dtype: object


In [6]:
ordinal_encoder = OrdinalEncoder(categories=[['not addicted', 'addicted']])
Category_encoded = ordinal_encoder.fit_transform(df[['addicted']])
df['addicted'] = Category_encoded
df.head()

,daily_screen_time,app_sessions,social_media_usage,gaming_time,notifications,night_usage,age,work_study_hours,stress_level,apps_installed,addicted
0,2,29,0,0,49,0,44,5,3,35,0.0
1,6,29,1,2,65,1,29,5,9,21,1.0
2,9,28,2,0,57,3,28,7,5,39,1.0
3,6,39,2,0,69,1,28,6,8,24,1.0
4,5,37,3,1,64,2,27,4,5,26,1.0


## **Preprocess Data**

Data is already mostly preprocessed. Just Preform some imputation and Standardization (Not Normalization)
Notice that the transformed data is actually stored in variables other than data (data_num_scaled, data_cat_scaled). This is because we will change the original data in the pipelining portion

In [7]:
print("Missing values (should be 0):\n", df.isnull().sum())

Missing values (should be 0):
 daily_screen_time     0
app_sessions          0
social_media_usage    0
gaming_time           0
notifications         0
night_usage           0
age                   0
work_study_hours      0
stress_level          0
apps_installed        0
addicted              0
dtype: int64


In [8]:

data_num = df[num_cols]
data_cat = df[cat_cols]

imputer_num = SimpleImputer(strategy='median')
data_num_imputed = imputer_num.fit_transform(data_num)

imputer_cat = SimpleImputer(strategy='most_frequent')
data_cat_imputed = imputer_cat.fit_transform(data_cat)


In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_num_scaled = scaler.fit_transform(data_num_imputed)


In [10]:
df_num_scaled = pd.DataFrame(data_num_scaled, columns=num_cols)
df_cat_imputed = pd.DataFrame(data_cat_imputed, columns=cat_cols)

# Combine numeric and categorical data
df_final = pd.concat([df_num_scaled, df_cat_imputed], axis=1)

# Optional: Restore original column order (if necessary)
df_final = df_final[df.columns]

In [11]:

df_final.describe()

,daily_screen_time,app_sessions,social_media_usage,gaming_time,notifications,night_usage,age,work_study_hours,stress_level,apps_installed,addicted
count,1.358900e+04,1.358900e+04,1.358900e+04,1.358900e+04,1.358900e+04,1.358900e+04,1.358900e+04,1.358900e+04,1.358900e+04,1.358900e+04,13589.000000
mean,-9.411855e-17,-1.691519e-16,5.647113e-17,7.790924e-17,1.192168e-16,-5.647113e-17,4.392199e-17,9.856303e-17,1.450994e-17,-2.284989e-16,0.503790
std,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,1.000037e+00,0.500004
min,-1.986138e+00,-2.975516e+00,-1.283179e+00,-1.041367e+00,-2.750269e+00,-1.046105e+00,-1.784553e+00,-2.886124e+00,-1.863898e+00,-2.974269e+00,0.000000
25%,-9.319584e-01,-6.801770e-01,-4.531563e-01,-1.041367e+00,-7.866498e-01,-1.046105e+00,-7.961937e-01,-4.721980e-01,-5.529212e-01,-7.688509e-01,0.000000
50%,1.222209e-01,-5.077283e-03,-4.531563e-01,-3.445536e-02,-7.974700e-02,1.033977e-02,-5.505838e-03,1.058724e-02,-1.159289e-01,-9.026065e-02,1.000000
75%,6.493105e-01,6.700225e-01,3.768668e-01,9.724562e-01,7.842453e-01,1.066785e+00,7.851820e-01,4.933725e-01,7.580558e-01,5.883296e-01,1.000000
max,4.338938e+00,4.045521e+00,4.526982e+00,5.000102e+00,3.376222e+00,5.292564e+00,2.168886e+00,3.872869e+00,2.506025e+00,4.659871e+00,1.000000


In [12]:
print("Missing values per column:\n", df_final.isnull().sum())
df_final.to_csv("addiction after scaling and imputation.csv", index=False)

Missing values per column:
 daily_screen_time     0
app_sessions          0
social_media_usage    0
gaming_time           0
notifications         0
night_usage           0
age                   0
work_study_hours      0
stress_level          0
apps_installed        0
addicted              0
dtype: int64


<hr>
<p>Copyright &copy; 2018 IBM Developer Skills Network. This notebook and its source code are released under the terms of the <a href="https://cognitiveclass.ai/mit-license/">MIT License</a>.</p>